# Описание Домашнего Задания

Вас пригласили на работу в коммерческую компанию, занимающуюся разработкой автоматизированных торговых агентов. Одной из первых ваших задач будет подготовка данных для дальнейшей обработки и построения модели. Пообщавшись с коллегами, вы узнали, что вам предстоит работать с несколькими типами активов: акциями из списка SnP500 и криптовалютами (BTC, ETH, SOL, XRP). Вам планируют поручить краткосрочную и среднесрочную торговлю.

Вам предлагается на основе предоставленной информации:

1) Создать git-репозиторий, где будет храниться исходный код вашего проекта. Если вы используете приватный репозиторий – дайте преподавателям курса доступ к нему, для возможности проверки ДЗ.
2) Добавить файл лицензии, который отражает ваш взгляд на конфиденциальность информации, которую вы подготовите в рамках данного курса.
3) Создать код на Python, который загрузит на ваш локальный компьютер данные о котировках ценных бумаг из списка SnP500 и котировки криптовалют (BTC, ETH, SOL, XRP).
4) Поскольку вам предстоит много работать с ними в дальнейшем, подготовьте автоматическое отображение графиков текущей ситуации.
5) Проверьте нет ли в данных пропусков или ошибок. Проанализируйте выбросы. Оцените, на самом ли деле это выбросы или реальные данные, с которыми предстоит работать.

# 1. Git-репозиторий

Если вы это читаете - значит уже сработало :)

<b>В целом по подходу:</b>
- репозиторий открытый, по ссылке https://github.com/PaulDok/ML-Finance-Bot;
- protected ветка main (стабильная версия после выполнения ДЗ), дополняется MR'ами из develop - этапы развития проекта;
- ветка develop - для разработки / выполнения ДЗ. Делать ли её protected - думаю пока я один contributor нет, иначе была бы запротекчена для тестирования перед релизом (~MR в main);
- в README.md - порядок настройки окружения для воспроизводимости. При личной непереносимости conda можно и через venv поднять, ключевое здесь то что зависимости в проекте управляются через Poetry;
- в отдельных нумерованных блокнотах (аналогичных этому) планирую выполнение формальных признаков ДЗ, в блокноте "./notebooks/HW_development.ipynb" - итеративную разработку инфраструктурных моментов (например, для ДЗ№1 - механизм кэширования данных в локальной БД);
- для большей интерактивности демонстраций (кроме EDA, пожалуй) предлагается использовать Streamlit, т.к.
    - этот фреймворк довольно хорошо подходит для прототипирования UI;
    - в него несложно переносить визуализации на популярных библиотеках построения графиков;
- когда дойдём до создания ботов, подумаю о том как именно отрефакторить, т.к. пока мне не ясен целевой вид (будем поднимать сервис? экспортировать pickle какого-то специфичного объекта? и т.п.)


Для демонстрации выполненного в ДЗ функционала понадобятся импорты в т.ч. реализованных функций, импортируем их:

In [1]:
# Относительные ссылки, включая импорты, относительно корневой папки проекта
import os

os.chdir(os.path.dirname(os.getcwd()))

import main
import logging

from src.core import utils

# initialize
logger = logging.getLogger()
# initialize config dict
config = main.main_launch()


# 2. Файл лицензии

Вообще я до конца не понимаю чем они отличаются, ориентировался при выборе подсказками Github

Поскольку проект учебный - выбрал лицензию MIT как максимально свободную

# 3. Код на Python для загрузки данных о котировках ценных бумаг S&P500 + крипты

В этом моменте не до конца понятно - S&P500 это только индекс или все его составляющие?

Поскольку в целевом виде предполагается автоматизированное в т.ч. с помощью ML решение + управление портфелем, предполагаю что нужны все составляющие, их беру с Wikipedia (https://en.wikipedia.org/wiki/List_of_S%26P_500_companies), см. config.get_default_tickers()

Загрузка данных в локальную базу (сейчас SQLite, при масштабировании наверное стоило бы поднять что-то интереснее) происходит следующим образом:

In [2]:
utils.update_tickers_data(config.TICKERS, config.START_DT, config.END_DT, config.INTERVAL)

[INFO   ] 2025-03-07@02:30:07: Made sure table in database for interval='1d' exists
[INFO   ] 2025-03-07@02:30:07: Checking already available data...
[INFO   ] 2025-03-07@02:30:07: 508 tickers have no data at all
[INFO   ] 2025-03-07@02:30:07: 0 tickers lack history in start part
[INFO   ] 2025-03-07@02:30:07: 0 tickers lack history in end part
[INFO   ] 2025-03-07@02:30:07: Updating unavailable tickers data...
[INFO   ] 2025-03-07@02:30:07: Downloading data using yfinance


YF.download() has changed argument auto_adjust default to True


[*********************100%***********************]  508 of 508 completed
[INFO   ] 2025-03-07@02:30:27: Downloaded data shape: (365, 2540)
[INFO   ] 2025-03-07@02:30:27: reshaped: (127342, 7)
[INFO   ] 2025-03-07@02:30:31: Data in caching DB updated


Вывод ячейки выше может меняться, т.к. при повторных запусках скрипт проверяет что из запрошенного (тикеры / интервал / временные границы) уже есть локально чтобы не тянуть заново через интернет

<i>NOTE:</i> вижу что есть баг в том что последние данные оно пытается взять даже если они уже есть, но как пофиксить сейчас не знаю (вероятно нужны манипуляции с datetime + interval), пока оставлю в качестве техдолга

# 4. Автоматическое отображение графиков

За автоматическим - добро пожаловать в Streamlit демонстрацию

В функционал на данный момент зашил:
- возможность отображения цены закрытия и объемов
- интерактив (за счет Plotly)
- возможность стандартизации цен для визуального сравнения нескольких тикеров (~визуализации их корреляции)

Для примера отобразим графики криптовалют

In [3]:
tickers = ["BTC-USD", "ETH-USD", "SOL-USD", "XRP-USD"]

In [4]:
# Все 4 тикера, Close + Volume
utils.get_data_and_draw_figure(
    tickers, config.START_DT, config.END_DT, config.INTERVAL
).show()

[INFO   ] 2025-03-07@02:30:31: Getting history from local cache DB...
[INFO   ] 2025-03-07@02:30:31: Got history of shape (1460, 7), 0 NaNs


In [5]:
# Все 4 тикера, только Close
utils.get_data_and_draw_figure(
    tickers, config.START_DT, config.END_DT, config.INTERVAL, draw_volume=False
).show()

[INFO   ] 2025-03-07@02:30:31: Getting history from local cache DB...
[INFO   ] 2025-03-07@02:30:31: Got history of shape (1460, 7), 0 NaNs


In [6]:
# Все 4 тикера, нормализованный Close + Volume
utils.get_data_and_draw_figure(
    tickers, config.START_DT, config.END_DT, config.INTERVAL, scale_price=True
).show()

[INFO   ] 2025-03-07@02:30:32: Getting history from local cache DB...
[INFO   ] 2025-03-07@02:30:32: Got history of shape (1460, 7), 0 NaNs


In [7]:
# Все 4 тикера, нормализованный Close
utils.get_data_and_draw_figure(
    tickers, config.START_DT, config.END_DT, config.INTERVAL, draw_volume=False, scale_price=True
).show()

[INFO   ] 2025-03-07@02:30:32: Getting history from local cache DB...
[INFO   ] 2025-03-07@02:30:32: Got history of shape (1460, 7), 0 NaNs


# 5. Анализ пропусков / ошибок / выбросов

По пункту "оценить, на самом ли деле это выбросы или реальные данные, с которыми предстоит работать": одно не противоречит другому.

Поскольку данные получены от доверенного источника, даже если цены значительно изменяются (например, у криптовалют на старте), это реальность с которой необходимо работать.

Что именно с выбросами делать (сглаживать / обрезать / интерполировать / оставлять их факт в качестве фич для моделей) - другой и отдельный вопрос.

Отдельный sanity check - данные приведены к текущим акциям, скачков в моменты сплитов не обнаружено (например, TSLA сплитовалась 2022-08-25 и 2020-08-31, AAPL - 2020-08-31, 2014-06-09 и еще несколько раз в прошлом, NVDA - 2024-06-10, 2021-07-20 и еще до 2007)

In [8]:
# Не нормируем Close, при этом графики цен растут монотонно (почти)
utils.get_data_and_draw_figure(
    ["TSLA", "AAPL", "NVDA"],
    "2010-01-01",
    "2025-03-01",
    "1d",
    update_cache=True,
    draw_volume=False,
    scale_price=False,
).show()

[INFO   ] 2025-03-07@02:30:32: Made sure table in database for interval='1d' exists
[INFO   ] 2025-03-07@02:30:32: Checking already available data...
[INFO   ] 2025-03-07@02:30:32: 0 tickers have no data at all
[INFO   ] 2025-03-07@02:30:32: 3 tickers lack history in start part
[INFO   ] 2025-03-07@02:30:32: 0 tickers lack history in end part
[INFO   ] 2025-03-07@02:30:32: Updating tickers lacking early history data...
[INFO   ] 2025-03-07@02:30:32: Downloading data using yfinance
[*********************100%***********************]  3 of 3 completed
[INFO   ] 2025-03-07@02:30:33: Downloaded data shape: (3566, 15)
[INFO   ] 2025-03-07@02:30:33: reshaped: (10576, 7)
[INFO   ] 2025-03-07@02:30:34: Data in caching DB updated
[INFO   ] 2025-03-07@02:30:34: Getting history from local cache DB...
[INFO   ] 2025-03-07@02:30:34: Got history of shape (11317, 7), 0 NaNs


In [9]:
# Да и с нормированием аналогично
utils.get_data_and_draw_figure(
    ["TSLA", "AAPL", "NVDA"],
    "2010-01-01",
    "2025-03-01",
    "1d",
    update_cache=False,
    draw_volume=False,
    scale_price=True,
).show()

[INFO   ] 2025-03-07@02:30:34: Getting history from local cache DB...
[INFO   ] 2025-03-07@02:30:34: Got history of shape (11317, 7), 0 NaNs


По проверке на наличие пропусков:

In [10]:
import pandas as pd

In [11]:
# Возьмём вообще все данные
all_data = utils.get_history(config.TICKERS, '2010-01-01', config.END_DT, config.INTERVAL, update_cache=True)
all_data["Date"] = pd.to_datetime(all_data["Date"])

[INFO   ] 2025-03-07@02:30:34: Made sure table in database for interval='1d' exists
[INFO   ] 2025-03-07@02:30:34: Checking already available data...
[INFO   ] 2025-03-07@02:30:34: 0 tickers have no data at all
[INFO   ] 2025-03-07@02:30:34: 508 tickers lack history in start part
[INFO   ] 2025-03-07@02:30:34: 508 tickers lack history in end part
[INFO   ] 2025-03-07@02:30:34: Updating tickers lacking early history data...
[INFO   ] 2025-03-07@02:30:34: Downloading data using yfinance
[*********************100%***********************]  508 of 508 completed
[ERROR  ] 2025-03-07@02:31:17: 
1 Failed download:
[ERROR  ] 2025-03-07@02:31:17: ['SW']: YFPricesMissingError('possibly delisted; no price data found  (1d 2010-01-01 -> 2024-07-08 00:00:00) (Yahoo error = "Data doesn\'t exist for startDate = 1262322000, endDate = 1720411200")')
[INFO   ] 2025-03-07@02:31:22: Downloaded data shape: (4766, 2541)
[INFO   ] 2025-03-07@02:31:22: reshaped: (1753073, 7)
[INFO   ] 2025-03-07@02:31:47: Updat

Первое что видно (с самого начала) - некоторые из тикеров которые пришли из Википедии не достаются вообще ('BF.B', 'BRK.B)

Поиск в интернете подсказывает, что вместо точек там должны стоять дефисы, исправлено в config.py


In [12]:
all_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1837701 entries, 84120 to 1837700
Data columns (total 7 columns):
 #   Column  Dtype         
---  ------  -----         
 0   Date    datetime64[ns]
 1   Ticker  object        
 2   Open    float64       
 3   Low     float64       
 4   High    float64       
 5   Close   float64       
 6   Volume  float64       
dtypes: datetime64[ns](1), float64(5), object(1)
memory usage: 112.2+ MB


In [13]:
# NaNов не обнаружено
all_data.isna().sum()

Date      0
Ticker    0
Open      0
Low       0
High      0
Close     0
Volume    0
dtype: int64

In [14]:
# Посмотрим за сколько дней в истории у тикеров есть данные
summary = all_data.groupby('Ticker').agg({'Date': ['min', 'max', 'count']})
summary.columns = ['Date_min', 'Date_max', 'count']
summary["days"] = (summary["Date_max"] - summary["Date_min"]).dt.days

In [15]:
# В целом дней больше чем точек - это логично учитывая что не каждый день - торговый.
# Это необходимо будет учитывать и экстраполировать в случаях, когда нам будет необходим непрерывный временной ряд
summary


,Date_min,Date_max,count,days
Ticker,,,,
A,2010-01-04,2025-03-05,3816,5539
AAPL,2010-01-04,2025-03-05,3816,5539
ABBV,2013-01-02,2025-03-05,3062,4445
ABNB,2020-12-10,2025-03-05,1062,1546
ABT,2010-01-04,2025-03-05,3816,5539
...,...,...,...,...
YUM,2010-01-04,2025-03-05,3816,5539
ZBH,2010-01-04,2025-03-05,3816,5539
ZBRA,2010-01-04,2025-03-05,3816,5539


In [16]:
# моментов с дублированными датами не обнаружено
summary[summary["days"] < (summary['count'] - 1)]

,Date_min,Date_max,count,days
Ticker,,,,
